In [1]:
import datetime as dt
import pandas as pd

from controller.db import get_db_engine

from loader.dart_loader import get_dart_api_loader, get_dart_fss_loader
from loader.fdr_loader import get_fdr_loader
from loader.pykrx_loader import get_pykrx_loader

In [2]:
# CFG

## dart_api_cfg
DART_API_CFG = {
    "year": 2023,
    # (11011 : 연간보고서) / (11012 : 반기보고서) / (11013 : 1분기보고서) / (11014 : 3분기보고서)
    "reprt_code": 11014,
}

## fdr_cfg
FDR_CFG = {
    "start": "20200101",
    # "end": dt.date.today().strftime("%Y%m%d"),
    "end": (dt.date.today() - dt.timedelta(days=1)).strftime("%Y%m%d"),
}
## pykrx_cfg
PYKRX_CFG = {
    "start": "20200101",
    # "end": dt.date.today().strftime("%Y%m%d"),
    "end": (dt.date.today() - dt.timedelta(days=1)).strftime("%Y%m%d"),
}

In [3]:
db_engine = get_db_engine()

In [4]:
# dart_fss
dart_fss_loader = get_dart_fss_loader()

## dart_info_df
dart_info_df = dart_fss_loader.get_info_df()
dart_info_df = dart_info_df[~(dart_info_df["stock_code"].isna())]
dart_info_df.to_sql(name="dart_info_df", con=db_engine, if_exists="replace", index=False)

Output()

Output()

Output()

3667

In [5]:
# dart_api_loader
dart_api_loader = get_dart_api_loader()

corp_codes = sorted(set(dart_info_df["corp_code"]))
## dart_fundamental_df
dart_fundamental_df = dart_api_loader.get_corps_fundamental_df(
    corp_codes=corp_codes, year=DART_API_CFG["year"], reprt_code=DART_API_CFG["reprt_code"]
)

dart_fundamental_df.to_sql(name="dart_fundamental_df", con=db_engine, if_exists="replace", index=False)

67355

In [6]:
# fdr_loader
fdr_loader = get_fdr_loader()

stock_codes = sorted(set(dart_info_df["stock_code"]))
## fdr_info_df
fdr_info_df = fdr_loader.get_krx_df()
fdr_info_df.to_sql(name="fdr_info_df", con=db_engine, if_exists="replace", index=False)

## fdr_ohlcv_df
fdr_ohlcv_df = fdr_loader.get_stocks_ohlcv_df(StockCodes=stock_codes, start=FDR_CFG["start"], end=FDR_CFG["end"])
fdr_ohlcv_df.to_sql(name="fdr_ohlcv_df", con=db_engine, if_exists="replace", index=False)

"000010" invalid symbol or has no data
"000110" invalid symbol or has no data
"000380" invalid symbol or has no data
"001150" invalid symbol or has no data
"001280" invalid symbol or has no data
"001490" invalid symbol or has no data
"001580" invalid symbol or has no data
"001590" invalid symbol or has no data
"001670" invalid symbol or has no data
"001830" invalid symbol or has no data
"001890" invalid symbol or has no data
"001980" invalid symbol or has no data
"002480" invalid symbol or has no data
"002850" invalid symbol or has no data
"002860" invalid symbol or has no data
"002930" invalid symbol or has no data
"002950" invalid symbol or has no data
"003020" invalid symbol or has no data
"003040" invalid symbol or has no data
"003050" invalid symbol or has no data
"003330" invalid symbol or has no data
"003590" invalid symbol or has no data
"003900" invalid symbol or has no data
"003930" invalid symbol or has no data
"004190" invalid symbol or has no data
"004510" invalid symbol o

2386461

In [7]:
# pykrx_loader
pykrx_loader = get_pykrx_loader()

## pykrx_info_df
KOSPI_trader_df = pykrx_loader.get_stock_trader_df(StockCode="KOSPI", start=PYKRX_CFG["start"], end=PYKRX_CFG["end"])
KOSPI_trader_df["Market"] = "KOSPI"

KOSDAQ_trader_df = pykrx_loader.get_stock_trader_df(StockCode="KOSDAQ", start=PYKRX_CFG["start"], end=PYKRX_CFG["end"])
KOSDAQ_trader_df["Market"] = "KOSDAQ"

pykrx_info_df = pd.concat([KOSDAQ_trader_df, KOSPI_trader_df], axis=0)
pykrx_info_df.reset_index(inplace=True)
pykrx_info_df.to_sql(name="pykrx_info_df", con=db_engine, if_exists="replace", index=False)

1924

In [8]:
# KRX Static
krx_kosdaq_df = pd.read_csv("./data/kosdaq_20231109.csv", encoding="cp949")
krx_kospi_df = pd.read_csv("./data/kospi_20231109.csv", encoding="cp949")

## krx_info_df
krx_info_df = pd.concat([krx_kosdaq_df, krx_kospi_df], axis=0)
krx_info_df.to_sql(name="krx_info_df", con=db_engine, if_exists="replace", index=False)

2637